In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.transforms
import seaborn as sns
import math
from math import sqrt, factorial
import re
from itertools import groupby
import itertools
import scipy
import random
from decimal import Decimal
from operator import itemgetter
import multiprocessing
from CustomFunctions import PCvisualization, DetailedBalance, segment_LLS, linear_cycle_utils
from pathlib import Path
from aicsimageio.writers.ome_tiff_writer import OmeTiffWriter
from aicsimageio.readers.czi_reader import CziReader
from CustomFunctions.MO_Threshold import MO_ma
import skimage
from scipy.spatial import distance
import numpy.ma as ma

In [6]:
#get directories and open separated datasets

mind = 'E:/Aaron/Combined_37C_Confocal_PCA_LLS_Apply/'
datadir = mind + 'Data_and_Figs/'
savedir = 'E:/Aaron/random_lls_smooth/' + 'singlecells/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

TotalFrame = pd.read_csv(datadir + 'Shape_Metrics_transitionPCbins.csv', index_col=0)


In [7]:
longcells = ['20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3_01']

In [4]:
############ define the cell to make a continuous movie of and some other info
ddir = '//10.158.28.35/TheriotLab_LLS7/Aaron/manual_crop_lls_decon/'
xy_buffer = 7
z_buffer = 7

cellname = '20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1_01'
cell = TotalFrame[TotalFrame.CellID==cellname]
cell = cell.sort_values('time').reset_index(drop=True)
lst = []
for x in cell.cell.to_list():
    x = x.split('_frame')[0]
    if x not in lst:
        lst.append(x)


In [9]:
for cellname in longcells:
    ############ define the cell to make a continuous movie of and some other info
    ddir = '//10.158.28.35/TheriotLab_LLS7/Aaron/manual_crop_lls_decon/'
    xy_buffer = 7
    z_buffer = 7

#     cellname = '20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1_01'
    cell = TotalFrame[TotalFrame.CellID==cellname]
    cell = cell.sort_values('time').reset_index(drop=True)
    lst = []
    for x in cell.cell.to_list():
        x = x.split('_frame')[0]
        if x not in lst:
            lst.append(x)

    ####### loop through all of the movies that captured the cell of interest
    ####### MODIFIED FROM segment_LLS.getbb_movie
    centroids = []
    imagelist = []
    framelist = []
    for l in lst:
        big = CziReader(ddir+l+'_deskewed_decon.czi')
        bigim = big.data
        orig_shape = big.shape
        cents = []
        reimlist = []
        croplist = []
        ### start by rescaling all of the images
        with multiprocessing.Pool(processes=60) as pool: 
            rescaled = pool.map(segment_LLS.quarter_scale, [i for i in bigim[:,1,:,:,:]])
        with multiprocessing.Pool(processes=60) as pool: 
            rescaledother = pool.map(segment_LLS.quarter_scale, [i for i in bigim[:,0,:,:,:]])
        ### get the shape of the rescaled to use later to leave out objects
        shape = np.array(rescaled).shape    
        #loop through time points to get bounding boxes
        cropdf = pd.DataFrame()
        for it, f in enumerate(rescaled):
            #mask the top pixels with signal
            if (f>100).any():
                mare = ma.masked_array(f, mask = f>np.percentile(f[f>100], 97))
                seg = MO_ma(mare, global_thresh_method='tri', object_minArea=200, local_adjust = 0.92)
                im_labeled, n_labels = skimage.measure.label(
                                          seg, background=0, return_num=True,  )

                im_props = skimage.measure.regionprops(im_labeled)
                tempdf = pd.DataFrame([])
                for count, prop in enumerate(im_props):
                    z,y,x = np.array(prop.centroid)
                    thebox = np.array(prop.bbox)
                    area = prop.area * 64
                    td = {'cell':count, 'z_min':thebox[0], 'y_min':thebox[1], 
                            'x_min':thebox[2],'z_max':thebox[3], 'y_max':thebox[4], 'x_max':thebox[5],
                           'z':z, 'y':y, 'x': x, 'z_range': seg.shape[-3], 'area':area}
                    #ensure only things that aren't on the edge are chosen
                    #and are big enough
                    if (td['z_min']>0) and (td['y_min']>0) and (td['x_min']>0) and (td['z_max']/4<shape[-3]) and (td['y_max']/4<shape[-2]) and (td['x_max']/4<shape[-1]) and (area>50000):
                        tempdf = tempdf.append(td, ignore_index=True)


                if len(tempdf)>0:
                    #if there's only one option pick that
                    if len(tempdf)==1:
                        cropdf = cropdf.append(tempdf, ignore_index=True)
                    #if first frame get the closest object to the center
                    elif (it==0):
                        dists = []
                        for p, row in tempdf.iterrows():
                            dists.append(distance.pdist([np.array(shape)[-3:][::-1]/2,
                                            row[['x','y','z']].values]))
                        cropdf = cropdf.append(tempdf.iloc[np.argmin(dists)], ignore_index=True)
                    #if it's any other frame get the closest object to the last pick
                    else:
                        dists = []
                        for p, row in tempdf.iterrows():
                            dists.append(distance.pdist([cropdf[['x','y','z']].iloc[-1].values,
                                            row[['x','y','z']].values]))
                        cropdf = cropdf.append(tempdf.iloc[np.argmin(dists)], ignore_index=True)
                #if there's no options fill the gap with nan
                else:
                    cropdf = cropdf.append({'cell':np.nan, 'z_min':np.nan, 'y_min':np.nan,
                            'x_min':np.nan,'z_max':np.nan, 'y_max':np.nan, 'x_max':np.nan,
                           'z':np.nan, 'y':np.nan, 'x': np.nan, 'z_range': np.nan, 'area':np.nan}, ignore_index=True)
            else:
                cropdf = cropdf.append({'cell':np.nan, 'z_min':np.nan, 'y_min':np.nan, 
                        'x_min':np.nan,'z_max':np.nan, 'y_max':np.nan, 'x_max':np.nan,
                       'z':np.nan, 'y':np.nan, 'x': np.nan, 'z_range': np.nan, 'area':np.nan}, ignore_index=True)


        ###aggregate cropping info from this particular image
        cropdf['x_min'] = int(max(0, cropdf['x_min'].min()-xy_buffer))
        cropdf['y_min'] = int(max(0, cropdf['y_min'].min()-xy_buffer))
        cropdf['z_min'] = int(max(0, cropdf['z_min'].min()-z_buffer))
        cropdf['z_max'] = int(min(cropdf['z_max'].max()+z_buffer, shape[-3]))
        cropdf['y_max'] = int(min(cropdf['y_max'].max()+xy_buffer, shape[-2])+1)
        cropdf['x_max'] = int(min(cropdf['x_max'].max()+xy_buffer, shape[-1])+1)
        mincrop = np.min(cropdf[['z_min','y_min','x_min']].values, axis = 0)
        maxcrop = np.max(cropdf[['z_max','y_max','x_max']].values, axis = 0)
        rescaledall = np.stack((rescaled,rescaledother))
        imagelist.append(rescaledall[:,:,
                                mincrop[0]:maxcrop[0],
                              mincrop[1]:maxcrop[1],
                              mincrop[2]:maxcrop[2]])
        centroids.append(cropdf[['z','y','x']].values-np.array([mincrop[0],mincrop[1],mincrop[2]]))
        framelist.append([l+'_frame_'+'{:03d}'.format(r) for r in range(len(cropdf))])
        print(f'finished cropping move {l}')
    ###pull all the images into one movie with minimal dimensions
    #first make sure there's no nan, if there is replace it with the centroid before it
    for i, l in enumerate(centroids):
            if np.any(np.isnan(l)):
                naninds = np.unique(np.where(np.isnan(l))[0])
                centroids[i][naninds] = [centroids[i][naninds.min()-1]]*len(naninds)
    #then get the relative positions of the first frames
    newc = []
    for u, c in enumerate(centroids):
        if u!=0:
            newc.append(centroids[u-1][-1]-c[0])
        else:
            newc.append(np.zeros(3))
    zeropos = np.stack(newc)
    zeropos = np.cumsum(zeropos,axis=0)
    zeropos = np.round(zeropos + abs(zeropos.min())+0.0001)
    shapes = [x.shape for x in imagelist]
    totalshape = np.round(np.max(zeropos + np.array(shapes)[:,-3:], axis = 0)-np.min(zeropos,axis=0)+0.001)
    finalimage = np.zeros((np.concatenate([[sum([x[1]for x in shapes])],[2],totalshape]).astype(int)))
    timecount = 0
    shifts = zeropos-np.min(zeropos,axis=0)
    for r in range(len(shapes)):
        s = shapes[r]
        z = shifts[r].astype(int)
        finalimage[timecount:timecount+s[1],
                   :,
                   z[-3]:z[-3]+s[-3],
                   z[-2]:z[-2]+s[-2],
                   z[-1]:z[-1]+s[-1],
                  ] = np.swapaxes(imagelist[r],0,1)
        timecount = timecount+s[1]

    indivdir = savedir + cellname +'/'
    if not os.path.exists(indivdir):
        os.makedirs(indivdir)
    #save full image
    OmeTiffWriter.save(finalimage.astype('uint16'),indivdir+f'{cellname}_full_movie.ome.tiff', dim_order='TCZYX')
    #save maximum intensity projection
    maxproj = np.max(finalimage, axis = 2)
    OmeTiffWriter.save(maxproj.astype('uint16'),indivdir+f'{cellname}_full_moviemax.ome.tiff', dim_order='TCYX')
    #save which frames from which videos were actually used
    pd.Series(framelist).to_csv(indivdir+f'{cellname}_framelist.csv')

finished cropping move 20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3-01-Subset-01
finished cropping move 20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3-02-Subset-01
finished cropping move 20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3-03-Subset-01
finished cropping move 20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3-04-Subset-01
finished cropping move 20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3-05-Subset-01
finished cropping move 20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3-06-Subset-01
finished cropping move 20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3-07-Subset-01


In [10]:
indivdir

'E:/Aaron/random_lls_smooth/singlecells/20240527_488_EGFP-CAAX_640_SPY650-DNA_cell3_01/'